<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W4/amazon_review_polarity_csv/cnn_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Using PyTorch

For some reason, the cost never goes below around 0.69. :(

In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device=torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

data=pd.read_csv("xab",encoding="utf-8")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data['label'],test_size=0.3)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=50
epochs=5
lr=1e-3

class fc_classifier(nn.Module):
    def __init__(self,n):
        super(fc_classifier,self).__init__()
        self.n=n
        self.network=nn.Sequential(
            nn.Linear(self.n,360),
            nn.ReLU(),
            nn.Linear(360,180),
            nn.ReLU(),
            nn.Linear(180,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.network(x)

class classifier(nn.Module):
    def __init__(self,n,kerns,pool_kerns,out_chans):
        super(classifier,self).__init__()
        self.kerns=kerns
        self.pool_kerns=pool_kerns
        self.convs=[]
        self.pools=[]
        self.stride=1
        self.n=n
        #self.embed=nn.Embedding(n,max_length)
        in_chan=1
        for i in range(len(kerns)):
            out_chan=out_chans[i]
            kern=kerns[i]
            pool_kern=pool_kerns[i]
            self.convs.append(nn.Conv1d(in_chan,out_chan,kern))
            self.compute_size(kern,self.stride)
            self.pools.append(nn.MaxPool1d(pool_kern))
            self.compute_size(pool_kern,pool_kern)
            in_chan=out_chan
        self.convs=nn.ModuleList(self.convs)
        self.pools=nn.ModuleList(self.pools)
        self.fc1=nn.Linear(self.n*out_chan,1)
        #self.fc2=nn.Linear(100,1)

    def compute_size(self,kernel,stride):
        self.n=(self.n-kernel)//stride+1
        return self.n

    def forward(self,x):
        #x=self.embed(x)
        for i in range(len(self.kerns)):
            x=nn.LeakyReLU()(self.convs[i](x))
            x=self.pools[i](x)
            x=nn.Dropout(0.2)(x)
        x=torch.flatten(x,1)
        #x=F.relu(self.fc1(x))
        #x=self.fc2(x)
        x=self.fc1(x)
        x=nn.Sigmoid()(x)
        return x

class corpus(Dataset):
    def __init__(self,corpus,label):
        self.corpus=corpus
        self.label=label
    def __len__(self):
        return len(self.corpus)
    def __getitem__(self,idx):
        return self.corpus[idx],self.label[idx]

def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y=x.to(device),y.to(device)
        #print(torch.sum(x[0]!=0))
        pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
        #pred=model(x)
        cost=loss_fn(pred.flatten(),y)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 10 == 0:
            cost, current = cost.item(), batch * bsize + len(x)
            print(f"cost: {cost:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x,y=x.to(device),y.to(device)
            pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
            #pred=model(x)
            #print(torch.round(pred.flatten()),y)
            test_loss += loss_fn(pred.flatten(), y).item()
            ncorrect = (torch.round(pred.flatten()) == y).sum().item()
            print(ncorrect)
            correct += ncorrect

    print(correct,size)
    print(f"Test Error: \n Accuracy: {(100*correct/size):>0.1f}%, Avg loss: {test_loss/size:>8f} \n")

text="ajbjdfowd"
#input =torch.from_numpy(tfidf.transform([text]).toarray()).float()
train_corpus_data=torch.from_numpy(train_corpus_tfidf.toarray()).float()
test_corpus_data=torch.from_numpy(test_corpus_tfidf.toarray()).float()
train_label_data=torch.from_numpy(train_label).float()
test_label_data=torch.from_numpy(test_label).float()

train_loader=DataLoader(corpus(train_corpus_data,train_label_data),batch_size=bsize)
test_loader=DataLoader(corpus(test_corpus_data,test_label_data),batch_size=bsize)

#input=input.reshape(1,input.shape[0],input.shape[1]
#c=classifier(train_corpus_data.shape[1],[2,3,4,5],[2,2,2,2],[6,16,24,32])
cnn_classifier=classifier(train_corpus_data.shape[1],[2],[2],[3]).to(device)
#input=input.reshape(1,input.shape[0],input.shape[1])
#print(c.forward(input))
dense_classifier=fc_classifier(train_corpus_data.shape[1]).to(device)

c=cnn_classifier

loss=nn.BCELoss()
#loss=nn.BCEWithLogitsLoss()
#optim=torch.optim.SGD(c.parameters(),lr=lr)
optim=torch.optim.Adam(c.parameters(),lr=lr)
#optim=torch.optim.ASGD(c.parameters(),lr=lr)

#print(torch.sum(train_label_data==1))
total_params = sum(p.numel() for p in c.parameters() if p.requires_grad)
print(f"Total params = {total_params}")
for epoch in range(epochs):
    train(train_loader,c,loss,optim)
test_loop(test_loader,c,loss)

Total params = 70810
cost: 0.700127  [   50/14000]
cost: 0.966084  [  550/14000]
cost: 0.798464  [ 1050/14000]
cost: 1.119800  [ 1550/14000]
cost: 0.830485  [ 2050/14000]
cost: 0.691831  [ 2550/14000]
cost: 0.660004  [ 3050/14000]
cost: 0.726754  [ 3550/14000]
cost: 0.714737  [ 4050/14000]
cost: 0.760426  [ 4550/14000]
cost: 0.818406  [ 5050/14000]
cost: 0.752084  [ 5550/14000]
cost: 0.810581  [ 6050/14000]
cost: 0.674271  [ 6550/14000]
cost: 0.811731  [ 7050/14000]
cost: 0.676185  [ 7550/14000]
cost: 0.708537  [ 8050/14000]
cost: 0.773656  [ 8550/14000]
cost: 0.697883  [ 9050/14000]
cost: 0.681019  [ 9550/14000]
cost: 0.835877  [10050/14000]
cost: 0.667464  [10550/14000]
cost: 0.875347  [11050/14000]
cost: 0.692823  [11550/14000]
cost: 0.947867  [12050/14000]
cost: 0.663875  [12550/14000]
cost: 0.863540  [13050/14000]
cost: 0.739048  [13550/14000]
cost: 0.813286  [   50/14000]
cost: 0.675583  [  550/14000]
cost: 0.629764  [ 1050/14000]
cost: 0.682934  [ 1550/14000]
cost: 0.659353  [ 2

# Keras

Yay, it works! (A bit of overfitting is happening, but at least the first few epochs look good.)

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection


data=pd.read_csv("xab",encoding="utf-8")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data["label"],test_size=0.4)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=32
epochs=8
lr=1e-5

features=train_corpus_tfidf.shape[1]
train_corpus_tfidf=train_corpus_tfidf.toarray()
test_corpus_tfidf=test_corpus_tfidf.toarray()
train_corpus_tfidf=train_corpus_tfidf.reshape(train_corpus_tfidf.shape[0],train_corpus_tfidf.shape[1],1)
test_corpus_tfidf=test_corpus_tfidf.reshape(test_corpus_tfidf.shape[0],test_corpus_tfidf.shape[1],1)

model=Sequential([InputLayer(shape=(features,1),batch_size=bsize),
                  Conv1D(3,2,activation="relu"),
                  MaxPooling1D(),
                  Dropout(0.2),
                  #Conv1D(32,16,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(3,2,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(36,4,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(18,8,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  Flatten(),
                  #Dense(100,activation="relu"),
                  #Dense(50,activation="relu"),
                  Dense(1,activation="sigmoid")
                  ])
#model=Sequential([InputLayer(shape=(features,)),
#                  Dense(100,activation="relu"),
#                  Dense(1,activation="sigmoid")
#                  ])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

model.fit(train_corpus_tfidf, train_label, epochs=epochs, batch_size=bsize, validation_data=(test_corpus_tfidf, test_label),shuffle=True)
eval_loss,eval_accuracy=model.evaluate(test_corpus_tfidf,test_label)
print(eval_loss,eval_accuracy)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (32, 47200, 3)         │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (32, 23600, 3)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 23600, 3)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 70800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │        70,801 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,810 (276.60 KB)

 Trainable params: 70,810 (276.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.6585 - loss: 0.6847 - val_accuracy: 0.7866 - val_loss: 0.6317
Epoch 2/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8402 - loss: 0.5876 - val_accuracy: 0.8267 - val_loss: 0.5157
Epoch 3/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8727 - loss: 0.4515 - val_accuracy: 0.8347 - val_loss: 0.4394
Epoch 4/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.8957 - loss: 0.3603 - val_accuracy: 0.8391 - val_loss: 0.3973
Epoch 5/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9168 - loss: 0.2960 - val_accuracy: 0.8414 - val_loss: 0.3749
Epoch 6/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9275 - loss: 0.2482 - val_accuracy: 0.8434 - val_loss: 0.3613
Epoch 7/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9346 - loss: 0.2169 - val_accuracy: 0.8472 - val_loss: 0.3550
Epoch 8/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9419 - loss: 0.1913 - val_accur